In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import multiprocessing
from sklearn.metrics import mean_absolute_error

pd.options.display.max_rows = 200

np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=100, suppress=True)

# 一 读入数据

In [2]:
# 训练集
train = pd.read_csv("../data/training_ratings.csv",usecols=[0,1,2])

In [3]:
# 测试集
test = pd.read_csv("../data/testing_ratings.csv",usecols=[0,1,2])

# 二 模型求解

In [4]:
# 用户-电影评分矩阵
train_matrix = train.pivot_table(index=["userId"], columns=["movieId"],values="rating")

In [5]:
# 用户-电影评分矩阵0（用0填充nan）
train_matrix0 = train_matrix.fillna(0)

In [6]:
# 每个用户打过分的电影列表
dict_key_user_value_movies = {}
for user in train_matrix.index:
    dict_key_user_value_movies[user]=train_matrix.loc[[user],:].dropna(axis=1).columns

In [7]:
# 每个电影评过分的用户列表
dict_key_movie_value_users = {}
for movie in train_matrix.columns:
    dict_key_movie_value_users[movie]=train_matrix.loc[:,movie].dropna().index

In [9]:
from sklearn.metrics.pairwise import pairwise_distances
# 用户相似度矩阵
user_similarity = pairwise_distances(train_matrix.fillna(0), metric='cosine')
# 物品相似度矩阵，没用到
item_similarity = pairwise_distances(train_matrix.fillna(0).T, metric='cosine')

In [10]:
user_similarity.shape

(610, 610)

In [11]:
item_similarity.shape

(9362, 9362)

In [12]:
# 辅助函数，若待预测电影在训练集没出现，那就没公共打分用户，所以返回提示码999
def func(x,c_movie):
    l = list(set(dict_key_movie_value_users[x['movieId']]).intersection(set(dict_key_movie_value_users[c_movie])))
    if l:
        return l
    else:
        return 999

# 保证评分一定在0.5~5.0之间，并且保留一位小数
def s_round(num):
    return round(max(0.5,min(5.0,num)),1)

# 保证评分是0.5 1.0 1.5 2.0这种步长为0.5的
def f_round(num):
    z = int(num)
    f = num - int(num)
    new_f = f
    if f<=0.25:
        new_f = 0
    elif (f>0.25 and f<0.745):
        new_f = 0.5
    elif f>=0.745:
        new_f = 1.0
    return z + new_f

# slopeone算法：首先找出当前用户已打分电影，然后分别找出这些电影已打分用户中有哪些是当前电影和已有电影有重叠的，然后对这个已打分电影计算一个权重
def predict(c_user, c_movie,train_matrix1,train_matrix,dict_key_user_value_movies,dict_key_movie_value_users,user_similarity):
    #先判断当前电影是否在训练集中出现过
    if c_movie not in train_matrix1.columns:
        t1 = time.time()
        pred_rating = train_matrix1.loc[[c_user],:].mean(axis=1)[c_user].round(1)
        t2 = time.time()
        # print(str(c_user)+'=='+ str(c_movie)+': '+str(pred_rating)+', time(s): '+str(t2-t1))
        return pred_rating,pred_rating,pred_rating,f_round(pred_rating) # c_user给其他电影的平均分
    else:
        c_user_list = dict_key_user_value_movies[c_user]
        t2 = time.time()
        df = pd.DataFrame(c_user_list)
        # print(df.values.flags)
        df['l'] = df.apply(lambda x: func(x,c_movie),axis=1)
        df = df[df['l']!=999] # 剔除l为空的行
        df['l_len'] = df.apply(lambda x: len(x['l']),axis=1)
        df['dev_m_list'] = df.apply(lambda x: train_matrix[x['movieId']][c_user], axis=1)
        df['m_dev_list'] = df.apply(lambda x :[(train_matrix[c_movie][u] - train_matrix[x['movieId']][u]) for u in x['l']],axis=1)
        # print(df['l_len'])
        df['bio_list'] = df.apply(lambda x :[(train_matrix[c_movie][u]*train_matrix[x['movieId']][u])/(len(dict_key_movie_value_users[x['movieId']])*len(dict_key_user_value_movies[u])) for u in x['l']],axis=1)
        df['bio'] = df.apply(lambda x: sum(x['bio_list']), axis=1)
        
        df['bio_u_list'] = df.apply(lambda x :[user_similarity[int(c_user)-1][u-1] for u in x['l']],axis=1)
        df['dev_list'] = df.apply(lambda x: sum(np.multiply(np.array(x['m_dev_list']),np.array(x['bio_u_list'])))/sum(x['bio_u_list']), axis=1)
        df['dev_old_list'] = df.apply(lambda x: (sum(x['m_dev_list'])/len(x['m_dev_list'])), axis=1)
#         df['dev_delta_list'] = df['dev_old_list']-df['dev_list'] #0.01级别
        #最原始
        pred_rating0 = ((df['dev_old_list'] + df['dev_m_list']).mean())
        #加权二部图对最后一步的电影贡献加权
        pred_rating1 = (((df['dev_old_list'] + df['dev_m_list'])*df['bio']).sum()/df['bio'].sum())
        #用户相似矩阵对一开始的单个电影加权
        pred_rating2 = (((df['dev_list'] + df['dev_m_list'])*df['bio']).sum()/df['bio'].sum())
        #对1取整和0.5
        pred_rating3 = f_round(pred_rating1)
        t3 = time.time()
        # print(str(c_user)+'--'+ str(c_movie)+': '+str(pred_rating3)+', t2(s): '+str(t3-t2))
        return s_round(pred_rating0),s_round(pred_rating1),s_round(pred_rating2),s_round(pred_rating3)

In [13]:
%%time
# 预测单条记录
predict(599,1291,train_matrix,train_matrix0,dict_key_user_value_movies,dict_key_movie_value_users,user_similarity)

CPU times: user 1.97 s, sys: 28.1 ms, total: 2 s
Wall time: 2.03 s


(3.5, 3.5, 3.4, 3.5)

In [14]:
# 将数据集按行分为8大块，进行并行计算
def parallelize_dataframe(data, func):
    partitions = 8
    data_split = np.array_split(data, partitions) #划分数据
    pool = multiprocessing.Pool(processes=partitions) 
    data = pd.concat(pool.map(func, data_split)) #处理每个数据块，然后合到一起
    pool.close()
    pool.join()
    return data

def work(data):
    data['pred'] = data.apply(lambda x: predict(x['userId'],x['movieId'],train_matrix,train_matrix0,dict_key_user_value_movies,dict_key_movie_value_users,user_similarity), axis=1)
    return data

In [15]:
%%time 
# 9min 18s
# 22min 33s
test = parallelize_dataframe(test, work)

CPU times: user 1.12 s, sys: 554 ms, total: 1.67 s
Wall time: 26min 17s


In [22]:
# 计算MAE得分
for i in range(4):
    test[f'pred{i}'] = test.apply(lambda x: x['pred'][i], axis=1)
    print(round(mean_absolute_error(test[f'pred{i}'], test['rating']),5))

0.67117
0.64963
0.64983
0.63841


# 保存结果

In [23]:
test.to_csv("../result/all_pred_testing_ratings.csv",index=None,float_format = '%.1f')